In [2]:
%matplotlib inline
%reset
from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
import scipy

import SchemDraw as schem # to draw schematics (install using 'pip install SchemDraw')
import SchemDraw.elements as e

import skrf as rf # scikit-rf (install using 'conda -c scikit-rf scikit-rf')

import sympy as s # sympy (install using 'conda -c sympy sympy')
from sympy import I # imaginary symbol

rf.stylely() # enable scikit-rf plot styles
s.init_printing(use_latex='mathjax') # print sympy equations in MathJAX (for Jupyter)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [24]:
def mag2db(mag):
    return np.multiply(20.0, np.log10(mag))
def mag2pdb(mag):
    return np.multiply(10.0, np.log10(mag))
def db2mag(db):
    return np.power(10, np.divide(db, 20.0))
def pdb2mag(pdb):
    return np.power(10, np.divide(pdb, 10.0))
def rad2deg(rad):
    return np.multiply(np.divide(rad, (2*np.pi)), 360.0)
def deg2rad(deg):
    return np.multiply(np.divide(deg, 360.0), 2*np.pi)
def dbm2v(dbm, R = 50):
    # P = V^2 / R
    power = 1e-3 * np.power(10, np.divide(dbm, 10))
    return np.sqrt(power * (R*2))    
def v2dbm(v, R = 50):
    power = np.divide(np.power(v, 2), R*2)
    return 10 * np.log10(power/1e-3)

In [3]:
# Single tone harmonic distortion
A, w1, t = s.symbols('A \omega_1 t', real=True)
a1, a2, a3 = s.symbols('a_1 a_2 a_3')
Si = A * s.cos(w1*t)
display(Si)
So = a1*Si + a2*(Si**2) + a3*(Si**3)
display(s.expand(s.fu(So, measure=lambda x: -x.count_ops())))

A⋅cos(\omega₁⋅t)

   3                      3                        2                        2 
3⋅A ⋅a₃⋅cos(\omega₁⋅t)   A ⋅a₃⋅cos(3⋅\omega₁⋅t)   A ⋅a₂⋅cos(2⋅\omega₁⋅t)   A ⋅
────────────────────── + ────────────────────── + ────────────────────── + ───
          4                        4                        2                2

                        
a₂                      
── + A⋅a₁⋅cos(\omega₁⋅t)
                        

In [5]:
# Two tone harmonic distortion
A, B, w1, w2, t = s.symbols('A B \omega_1 \omega_2 t', real=True)
a1, a2, a3 = s.symbols('a_1 a_2 a_3')
Si = A * s.cos(w1*t) + B * s.cos(w2*t)
display(Si)
So = a1*Si + a2*(Si**2) + a3*(Si**3)
So = s.expand(s.fu(So, measure=lambda x: -x.count_ops()))
display(So)

A⋅cos(\omega₁⋅t) + B⋅cos(\omega₂⋅t)

   3                      3                          2                        
3⋅A ⋅a₃⋅cos(\omega₁⋅t)   A ⋅a₃⋅cos(3⋅\omega₁⋅t)   3⋅A ⋅B⋅a₃⋅cos(\omega₂⋅t)   3
────────────────────── + ────────────────────── + ──────────────────────── + ─
          4                        4                         2                

  2                                        2                                  
⋅A ⋅B⋅a₃⋅cos(2⋅\omega₁⋅t - \omega₂⋅t)   3⋅A ⋅B⋅a₃⋅cos(2⋅\omega₁⋅t + \omega₂⋅t)
───────────────────────────────────── + ──────────────────────────────────────
                 4                                        4                   

    2                        2           2                          2         
   A ⋅a₂⋅cos(2⋅\omega₁⋅t)   A ⋅a₂   3⋅A⋅B ⋅a₃⋅cos(\omega₁⋅t)   3⋅A⋅B ⋅a₃⋅cos(\
 + ────────────────────── + ───── + ──────────────────────── + ───────────────
             2                2                2                              

                               2                 

In [10]:
So_1805_1810_mix = So.subs([(w1, 1810), (w2, 1805)])
display(So_1805_1810_mix)

   3                   3                     2                       2        
3⋅A ⋅a₃⋅cos(1810⋅t)   A ⋅a₃⋅cos(5430⋅t)   3⋅A ⋅B⋅a₃⋅cos(1805⋅t)   3⋅A ⋅B⋅a₃⋅co
─────────────────── + ───────────────── + ───────────────────── + ────────────
         4                    4                     2                       4 

               2                     2                   2           2        
s(1815⋅t)   3⋅A ⋅B⋅a₃⋅cos(5425⋅t)   A ⋅a₂⋅cos(3620⋅t)   A ⋅a₂   3⋅A⋅B ⋅a₃⋅cos(
───────── + ───────────────────── + ───────────────── + ───── + ──────────────
                      4                     2             2               4   

               2                       2                                      
1800⋅t)   3⋅A⋅B ⋅a₃⋅cos(1810⋅t)   3⋅A⋅B ⋅a₃⋅cos(5420⋅t)                       
─────── + ───────────────────── + ───────────────────── + A⋅B⋅a₂⋅cos(5⋅t) + A⋅
                    2                       4                                 

                                         3       

In [39]:
# Work out IIP cascade stuff from discussion to verify
# Discussion slide 4 (IIP 2 cascade)
VIIP_2A = dbm2v(0, R=50)
VIIP_2B = dbm2v(10, R=50)
Av = db2mag(20)
VIIP_2 = (1/VIIP_2A + Av/VIIP_2B)**-1
IIP_2 = v2dbm(VIIP_2,R=50)
print(VIIP_2A, VIIP_2B, Av, VIIP_2, IIP_2)

VIIP_2A = dbm2v(0, R=100)
VIIP_2B = dbm2v(10, R=10)
Av = db2mag(20)
VIIP_2 = (1/VIIP_2A + Av/VIIP_2B)**-1
IIP_2 = v2dbm(VIIP_2,R=100)
print(VIIP_2A, VIIP_2B, Av, VIIP_2, IIP_2)
print('Everything checks out!')

0.316227766017 1.0 10.0 0.0759746926648 -12.3866209613
0.4472135955 0.4472135955 10.0 0.0406557814091 -20.8278537032
Everything checks out!


In [49]:
# Discussion slide 5 (IIP3 cascade)
VIIP_3A = dbm2v(-10, R=50)
VIIP_3B = dbm2v(-20, R=50)
Av = np.sqrt((50/50) * pdb2mag(20))
VIIP_3 = np.sqrt((1/(VIIP_3A**2) + (Av**2)/(VIIP_3B**2))**-1)
IIP_3 = v2dbm(VIIP_3, R=50)
print(VIIP_3A, VIIP_3B, Av, VIIP_3, IIP_3)

VIIP_3A = dbm2v(-10, R=100)
VIIP_3B = dbm2v(-20, R=10)
Av = np.sqrt((100/10) * pdb2mag(20))
VIIP_3 = np.sqrt((1/(VIIP_3A**2) + (Av**2)/(VIIP_3B**2))**-1)
IIP_3 = v2dbm(VIIP_3, R=100)
print(VIIP_3A, VIIP_3B, Av, VIIP_3, IIP_3)
print('Everything checks out!')

0.1 0.0316227766017 10.0 0.00316069770621 -40.0043407748
0.141421356237 0.0141421356237 31.6227766017 0.000447211359449 -60.0000434292
Everything checks out!


In [60]:
# Problem 1a
VIIP_3A = dbm2v(50, R=50)
VIIP_3B = dbm2v(10, R=50)
Av = np.sqrt(pdb2mag(-1))
VIIP_3 = np.sqrt((1/(VIIP_3A**2) + (Av**2)/(VIIP_3B**2))**-1)
IIP_3 = v2dbm(VIIP_3, R=50)
print(VIIP_3A, VIIP_3B, Av, VIIP_3, IIP_3)

a3 = Av / (VIIP_3**2 * (3/4))
print(a3)
print((a3 *3 / 4) * dbm2v(-33, R=50) * (dbm2v(-27, R=50)**2))
dbm2v(-109, R=50)

100.0 1.0 0.891250938134 1.12194783409 10.9994532901
0.944046545971
1.00012589254e-06


1.1220184543e-06